In [1]:
"""
Preamble for most code and jupyter notebooks
@author: tobinsouth
@notebook date: 28 Oct 2021
"""

import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns
import math, string, re, pickle, json, os, sys, datetime, itertools
from collections import Counter
from tqdm import tqdm

# Set panda's options
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 120)

# Better graphics
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.style.use('seaborn-poster')

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch

class StaysDataset(Dataset):
    """Loads in stayz dataset as zipped csv files"""

    def __init__(self, root_dir):
        from glob import glob
        self.root_dir = root_dir
        self.all_csvs = glob(root_dir+'/*.csv.gz')
        stays = pd.concat([pd.read_csv(csv, nrows = 100000) for csv in self.all_csvs])

        self.all_users = list(stays['user'].unique())
        self.grouped_users = stays.groupby('user')
        self.user_homes = dict(self.grouped_users['GEOID_home'].unique())
        self.grouped_stays = self.grouped_users['GEOID']
        self.all_geoid = list(set(list(stays['GEOID'].unique()) + [l.item() for l in self.user_homes.values()]))
        self.all_geoid_mapping = dict(zip(self.all_geoid, range(1,len(self.all_geoid)+1)))

        # We could also truncate each time they leave home?


    def __len__(self):
        return len(self.all_users)

    def __getitem__(self, idx):
        """Get item from grouped frame"""
        user = self.all_users[idx]
        user_stays_seq = self.grouped_stays.get_group(user).to_list()
        user_home = self.user_homes[user].item()
        user_stays_seq = [self.all_geoid_mapping[user_home]] + [self.all_geoid_mapping[geoid] for geoid in user_stays_seq]
        user_stays_seq = torch.tensor(user_stays_seq, dtype=torch.long)
        return user_stays_seq


root_dir = '../data/'
staysDataset = StaysDataset(root_dir)

from torch.nn.utils.rnn import pad_sequence

dataloader = DataLoader(staysDataset, batch_size=1, shuffle=True, 
    collate_fn=lambda batch: pad_sequence(batch, batch_first=True, padding_value=0))

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [4]:

class SentEnc(nn.Module):
    def __init__(self, num_locations, hidden_size, dropout=0):
        super(SentEnc, self).__init__()
        self.embedding = nn.Embedding(num_locations, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, dropout=dropout, batch_first=True)
        self.linear =  nn.Linear(hidden_size, num_locations)      

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        loc_space = self.linear(lstm_out)
        return loc_space

In [5]:
HIDDEN_SIZE = 64
batch_size = 32
num_layers = 1
num_epochs = 3 
criterion = nn.CrossEntropyLoss()
dropout = 0

lstm = SentEnc(len(staysDataset.all_geoid), HIDDEN_SIZE, dropout)
optimizer = torch.optim.Adam(lstm.parameters()) 

# Training LSTM next step prediction on sequences
for epoch in range(num_epochs):
    for seq_batch in dataloader:
        seq_batch = seq_batch.to(device)
        optimizer.zero_grad()
        lstm_out = lstm(seq_batch)
        loss = criterion(lstm_out, seq_batch)
        loss.backward()
        optimizer.step()
        print(loss)

RuntimeError: Input, output and indices must be on the current device

In [8]:
for seq_batch in dataloader:
    break

In [9]:
print(seq_batch.shape)
x = lstm.embedding(seq_batch)
print(x.shape)
lstm_out, _ = lstm.lstm(x)
lstm.linear(x)

torch.Size([1, 436])
torch.Size([1, 436, 64])


tensor([[[ 0.3582,  0.6921, -0.5455,  ..., -0.1627, -0.1247,  0.6435],
         [ 0.6742, -0.6416, -1.3599,  ..., -0.0328,  0.0389, -0.9344],
         [ 0.3582,  0.6921, -0.5455,  ..., -0.1627, -0.1247,  0.6435],
         ...,
         [-0.3349,  0.2405, -0.7366,  ...,  0.7629,  0.5154, -0.7340],
         [ 0.0141,  0.9837, -0.9106,  ...,  0.5932, -0.0276, -0.3215],
         [ 0.3582,  0.6921, -0.5455,  ..., -0.1627, -0.1247,  0.6435]]],
       grad_fn=<AddBackward0>)

In [10]:
x

tensor([[[ 8.8092e-04,  1.3875e+00,  8.1915e-02,  ...,  9.6235e-01,
           2.7631e-01,  6.8436e-01],
         [ 1.0390e+00,  4.6175e-01,  3.3525e-01,  ..., -5.0176e-01,
          -2.7594e-01, -7.5815e-01],
         [ 8.8092e-04,  1.3875e+00,  8.1915e-02,  ...,  9.6235e-01,
           2.7631e-01,  6.8436e-01],
         ...,
         [-2.0933e-01, -6.8803e-03, -7.7174e-01,  ..., -7.1076e-01,
          -1.6150e+00,  9.9162e-01],
         [-1.7129e+00, -8.0305e-02,  2.0688e-01,  ..., -4.4951e-01,
           5.5182e-01,  3.7514e-01],
         [ 8.8092e-04,  1.3875e+00,  8.1915e-02,  ...,  9.6235e-01,
           2.7631e-01,  6.8436e-01]]], grad_fn=<EmbeddingBackward>)

In [10]:
from glob import glob
all_csvs = glob(root_dir+'/*.csv.gz')
stays = pd.concat([pd.read_csv(csv, nrows = 100000) for csv in all_csvs])

In [35]:
for user, seq in stays.groupby('user')['GEOID']:
    seq = torch.tensor([staysDataset.all_geoid_mapping[geoid] for geoid in seq]).reshape(1,-1)
    break